In [70]:
from itertools import product
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import seaborn as sns 
import matplotlib.pylab as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as path_effects
import json
import pingouin as pg

In [71]:
import sklearn
print(sklearn.__version__)

0.24.2


In [72]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


In [73]:
filter = "rare"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]


In [74]:
SEs["SIDER"].shape

(1344, 5481)

In [75]:
SEs["OFFSIDES"].shape

(2730, 6172)

In [76]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]

In [77]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    # "RF":[C], 
    # "BRF":[C]
}

In [78]:
SEs_name = "OFFSIDES"
metrice = "AUPR"

In [79]:
hyperpars = {}
hyperpars["nested_cv"] = {}
hyperpars["cv"] = {}
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

In [80]:
hyperpars["nested_cv"]["SKR"] = {}
hyperpars["nested_cv"]["SKR"]["target"] = [
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.5, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["enzyme"] = [
    (1, 0.9, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["Chem"] = [
    (1, 0.6, 10, 100),
    (1, 0.5, 10, 100),
    (1, 0.5, 10, 100),
    (10, 0.2, 10, 100),
    (10, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["DGI"] = [
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["transporter"] = [
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["pathway"] = [
    (0.01, 0.4, 10, 100),
    (0.01, 0.4, 10, 100),
    (0.01, 0.3, 10, 100),
    (0.01, 0.4, 10, 100),
    (0.01, 0.4, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["indication"] = [
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    ]
hyperpars["cv"]["SKR"] = {}
hyperpars["cv"]["SKR"]["target"] = (100, 0.1, 10, 100)
hyperpars["cv"]["SKR"]["enzyme"] = (1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["Chem"] = (1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["DGI"] =(0.1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["transporter"] = (0.01, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["pathway"] = (100, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["indication"] = (0.1, 0.9, 10, 100)


In [81]:
hyperpars["nested_cv"]["KRR"] = {}
hyperpars["nested_cv"]["KRR"]["target"] = [
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["enzyme"] = [
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["Chem"] = [
    (10, 10),
    (10, 10),
    (10, 10),
    (10, 10),
    (10, 10),
    ]
hyperpars["nested_cv"]["KRR"]["DGI"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["transporter"] = [
    (0.1, 10),
    (1, 10),
    (1, 10),
    (0.01, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["pathway"] = [
    (0.1, 10),
    (0.01, 10),
    (0.1, 10),
    (0.01, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["indication"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    ]
hyperpars["cv"]["KRR"] = {}
hyperpars["cv"]["KRR"]["target"] = (100, 10)
hyperpars["cv"]["KRR"]["enzyme"] = (100, 10)
hyperpars["cv"]["KRR"]["Chem"] = (0.1, 100)
hyperpars["cv"]["KRR"]["DGI"] = (0.1, 100)
hyperpars["cv"]["KRR"]["transporter"] = (0.01, 100)
hyperpars["cv"]["KRR"]["pathway"] = (100, 100)
hyperpars["cv"]["KRR"]["indication"] = (1, 100)

In [82]:
hyperpars["nested_cv"]["Naive"] = {}
hyperpars["nested_cv"]["Naive"]["target"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["enzyme"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["Chem"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["DGI"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["transporter"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["pathway"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["indication"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["cv"]["Naive"] = {}
hyperpars["cv"]["Naive"]["target"] = ()
hyperpars["cv"]["Naive"]["enzyme"] = ()
hyperpars["cv"]["Naive"]["Chem"] =()
hyperpars["cv"]["Naive"]["DGI"] = ()
hyperpars["cv"]["Naive"]["transporter"] = ()
hyperpars["cv"]["Naive"]["pathway"] = ()
hyperpars["cv"]["Naive"]["indication"] = ()

In [83]:
hyperpars["nested_cv"]["VKR"] = {}
hyperpars["nested_cv"]["VKR"]["target"] = [
    (10, 0.1, 15),
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.01, 15),
    ]
hyperpars["nested_cv"]["VKR"]["enzyme"] = [
    (10, 1, 15),
    (10, 0.01, 5),
    (10, 0.01, 10),
    (10, 1, 10),
    (10, 1, 10),
    ]
hyperpars["nested_cv"]["VKR"]["Chem"] = [
    (10, 10, 15),
    (10, 1, 15),
    (10, 1, 15),
    (10, 10, 15),
    (10, 10, 15),
    ]
hyperpars["nested_cv"]["VKR"]["DGI"] = [
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    ]
hyperpars["nested_cv"]["VKR"]["transporter"] = [
    (10, 0.1, 10),
    (10, 0.1, 15),
    (10, 0.1, 10),
    (10, 0.01, 10),
    (10, 0.01, 5),
    ]
hyperpars["nested_cv"]["VKR"]["pathway"] = [
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    ]
hyperpars["nested_cv"]["VKR"]["indication"] = [
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    ]
hyperpars["cv"]["VKR"] = {}
hyperpars["cv"]["VKR"]["target"] = (10, 100, 15)
hyperpars["cv"]["VKR"]["enzyme"] = (10, 100, 15)
hyperpars["cv"]["VKR"]["Chem"] =(100, 0.1, 15)
hyperpars["cv"]["VKR"]["DGI"] = (100, 0.01, 15)
hyperpars["cv"]["VKR"]["transporter"] = (100, 0.01, 15)
hyperpars["cv"]["VKR"]["pathway"] = (100, 100, 15)
hyperpars["cv"]["VKR"]["indication"] = (100, 100, 15)

In [84]:
hyperpars["nested_cv"]["LNSM_RLN"] = {}
hyperpars["nested_cv"]["LNSM_RLN"]["target"] = [
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["enzyme"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 100),
    (0.1, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["Chem"] = [
    (0.9, 100),
    (0.9, 100),
    (0.4, 100),
    (0.4, 100),
    (0.4, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["DGI"] = [
    (0.3, 100),
    (0.3, 100),
    (0.3, 100),
    (0.3, 100),
    (0.3, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["transporter"] = [
    (0.1, 10),
    (0.2, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["pathway"] = [
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["indication"] = [
    (0.7, 100),
    (0.7, 100),
    (0.7, 100),
    (0.8, 100),
    (0.8, 100)
    ]
hyperpars["cv"]["LNSM_RLN"] = {}
hyperpars["cv"]["LNSM_RLN"]["target"] = (0.1, 10)
hyperpars["cv"]["LNSM_RLN"]["enzyme"] = (0.1, 100)
hyperpars["cv"]["LNSM_RLN"]["Chem"] = (0.8, 100)
hyperpars["cv"]["LNSM_RLN"]["DGI"] = (0.2, 100)
hyperpars["cv"]["LNSM_RLN"]["transporter"] = (0.1, 10)
hyperpars["cv"]["LNSM_RLN"]["pathway"] = (0.1, 100)
hyperpars["cv"]["LNSM_RLN"]["indication"] = (0.5, 100)

In [85]:
hyperpars["nested_cv"]["LNSM_jaccard"] = {}
hyperpars["nested_cv"]["LNSM_jaccard"]["target"] = [
    (0.1,), 
    (0.2,), 
    (0.1,), 
    (0.1,), 
    (0.3,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["enzyme"] = [
    (0.8,), 
    (0.4,), 
    (0.7,), 
    (0.1,), 
    (0.9,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["Chem"] = [
    (0.1,), 
    (0.8,), 
    (0.3,), 
    (0.4,), 
    (0.8,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["DGI"] = [
    (0.2,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["transporter"] = [
    (0.7,),
    (0.3,),
    (0.3,),
    (0.6,),
    (0.4,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["pathway"] = [
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["indication"] = [
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["cv"]["LNSM_jaccard"] = {}
hyperpars["cv"]["LNSM_jaccard"]["target"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["enzyme"] = (0.4,)
hyperpars["cv"]["LNSM_jaccard"]["Chem"] = (0.8,)
hyperpars["cv"]["LNSM_jaccard"]["DGI"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["transporter"] = (0.4,)
hyperpars["cv"]["LNSM_jaccard"]["pathway"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["indication"] = (0.4,)

In [86]:

# # Open and read the JSON file
# with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
#     hyperpars = json.load(xml_file)

In [87]:
for method in methods:
    # # method = "SKR"
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    # method = "SKR"
    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

using feature target
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  45
first few training idx:  [ 35 107 184 194 251 328 410 461 578 653]
first few testing idx:  [224 236 273 301 306 322 406 434 435 585]
--- tuning end ---
target size: 133
------ best hyper pars:  (0.01, 0.6, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.06660713799659151
AUROCperdrug: 0.763499214292935
AUPR+AUROCperdrug: 0.8301063522895265
AUPR: 0.05529583154927727
AUROC: 0.7158406525058227
AUPR+AUROC: 0.7711364840551
-----------
Fold: 1
number of hyperpars combination:  45
first few training idx:  [224 236 273 301 306 322 406 434 435 585]
first few testing idx:  [ 35 107 184 194 251 328 410 461 578 653]
--- tuning end ---
target size: 133
------ best hyper pars:  (0.01, 0.6, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.045205174465070705
AUROCperdrug: 0.7311265260518489
AUPR+AUROCperdrug

In [88]:
with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
   json.dump(hyperparsOut, xml_file)
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

In [89]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

In [90]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            0.814269   0.018117          0.871404   
            KRR            0.805827   0.042154          0.862769   
            VKR            0.820921   0.027405          0.805101   
            Naive          0.767696   0.010109          0.804008   
            LNSM_RLN       0.808951   0.024936          0.825698   
            LNSM_jaccard   0.568672   0.059902          0.613726   
Chem        SKR            0.754919   0.004893          0.799146   
            KRR            0.760468   0.011330          0.799096   
            VKR            0.748777   0.023003          0.763515   
            Naive          0.768181   0.008198          0.791657   
            LNSM_RLN       0.612543   0.018600          0.635348   
            LNSM_jaccard   0.494104   0.064808          0.509939   
DGI         SKR            0.795032   0.032403          0.766325   
            KRR            0.788402   0.033184          0.754145   
            VKR            0.822825   0.056709          0.737703   
            Naive          0.763230   0.004684          0.791271   
            LNSM_RLN       0.508130   0.025001          0.498205   
            LNSM_jaccard   0.584726   0.072568          0.586484   
indication  SKR            0.828867   0.026429          0.798170   
            KRR            0.822033   0.027201          0.786478   
            VKR            0.858318   0.033817          0.755545   
            Naive          0.766308   0.008823          0.786497   
            LNSM_RLN       0.621617   0.044889          0.627211   
            LNSM_jaccard   0.830529   0.043140          0.849507   
target      SKR            0.762840   0.010292          0.808843   
            KRR            0.774719   0.013200          0.806249   
            VKR            0.762659   0.022568          0.768708   
            Naive          0.764287   0.007091          0.788572   
            LNSM_RLN       0.543575   0.051299          0.562496   
            LNSM_jaccard   0.521428   0.049517          0.463201   
transporter SKR            0.758890   0.015971          0.781509   
            KRR            0.742939   0.036255          0.758639   
            VKR            0.750352   0.031282          0.746489   
            Naive          0.779139   0.008246          0.800313   
            LNSM_RLN       0.713684   0.041494          0.697180   
            LNSM_jaccard   0.502103   0.018276          0.496839   
enzyme      SKR            0.761056   0.019546          0.766486   
            KRR            0.756048   0.021462          0.756548   
            VKR            0.765077   0.035166          0.743081   
            Naive          0.764574   0.003980          0.788434   
            LNSM_RLN       0.736784   0.042994          0.694325   
            LNSM_jaccard   0.522075   0.028632          0.506503   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.013377  0.741110  0.020481  0.073159   
            KRR                   0.029783  0.731458  0.041290  0.074368   
            VKR                   0.018085  0.755380  0.018511  0.065542   
            Naive                 0.008040  0.728631  0.003547  0.039066   
            LNSM_RLN              0.013919  0.734855  0.017131  0.074096   
            LNSM_jaccard          0.047646  0.538100  0.058566  0.030572   
Chem        SKR                   0.009303  0.714800  0.012575  0.040120   
            KRR                   0.007808  0.721034  0.016006  0

In [91]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_{filter}.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            0.767661          0.813291  0.717659  0.050001   
            KRR            0.767345          0.811571  0.717364  0.049981   
            VKR            0.763138          0.778752  0.713824  0.049315   
            Naive          0.770548          0.810164  0.719736  0.050811   
            LNSM_RLN       0.771845          0.770710  0.685184  0.086660   
            LNSM_jaccard   0.634257          0.625775  0.584585  0.049671   
Chem        SKR            0.798219          0.822046  0.741910  0.056309   
            KRR            0.780916          0.805757  0.735871  0.045044   
            VKR            0.770150          0.770245  0.727403  0.042748   
            Naive          0.773802          0.797056  0.730914  0.042888   
            LNSM_RLN       0.563883          0.581609  0.531000  0.032883   
            LNSM_jaccard   0.529088          0.533717  0.504267  0.024821   
DGI         SKR            0.833476          0.835969  0.761271  0.072205   
            KRR            0.772959          0.795644  0.730050  0.042909   
            VKR            0.807456          0.762960  0.749578  0.057877   
            Naive          0.759248          0.791190  0.721135  0.038113   
            LNSM_RLN       0.508657          0.523244  0.466457  0.042200   
            LNSM_jaccard   0.730844          0.693912  0.685904  0.044940   
indication  SKR            0.849747          0.869123  0.765105  0.084642   
            KRR            0.775213          0.803894  0.730775  0.044438   
            VKR            0.759624          0.771028  0.718291  0.041332   
            Naive          0.768696          0.798428  0.725313  0.043383   
            LNSM_RLN       0.650654          0.663755  0.566050  0.084604   
            LNSM_jaccard   0.513616          0.463805  0.485580  0.028036   
target      SKR            0.761298          0.800403  0.723482  0.037815   
            KRR            0.761802          0.800664  0.723754  0.038048   
            VKR            0.752825          0.766903  0.716564  0.036261   
            Naive          0.761134          0.795300  0.723270  0.037864   
            LNSM_RLN       0.650924          0.586719  0.593741  0.057183   
            LNSM_jaccard   0.583458          0.515137  0.547420  0.036038   
transporter SKR            0.783129          0.806742  0.732801  0.050327   
            KRR            0.779461          0.800410  0.730588  0.048873   
            VKR            0.771069          0.780166  0.723499  0.047570   
            Naive          0.776134          0.799114  0.727841  0.048293   
            LNSM_RLN       0.775453          0.717381  0.724412  0.051041   
            LNSM_jaccard   0.560009          0.536826  0.529118  0.030891   
enzyme      SKR            0.786742          0.803997  0.738925  0.047817   
            KRR            0.793786          0.802120  0.743106  0.050680   
            VKR            0.787551          0.780531  0.738128  0.049423   
            Naive          0.774607          0.801313  0.731106  0.043501   
            LNSM_RLN       0.817270          0.724952  0.755108  0.062161   
            LNSM_jaccard   0.501356          0.520768  0.477453  0.023903   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.760587    0.052704  
            KRR              0.759049    0.052522  
            VKR              0.727959    0.050793  
            Naive            0.757912    0.052252  
            LNSM_RLN         0.676270    0.094440  
            LNSM_jaccard     0.555283    0.070492  
Chem        SKR              0.766944    0.055103  
            KRR              0.758818    0.0

In [92]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')